In [1]:
urls = ['https://www.msnbc.com/transcripts/show/rachel-maddow-show', 'https://www.msnbc.com/transcripts/show/rachel-maddow-show?page=2',
       'https://www.msnbc.com/transcripts/show/rachel-maddow-show?page=3']

In [2]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains



In [3]:
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [4]:
url_list = []
transcript_list = []
time_list = []

for url in urls: 
    # move to that page, pull the page_source
    driver.get(url)
    page_source = driver.page_source
    
    soup = BeautifulSoup(page_source)
    url_soup = soup.find_all('a', href=re.compile('.*transcripts.*'))
    
    
    for link in url_soup: 
        link = str(link.get('href'))
        if link == 'https://www.msnbc.com/transcripts':
            pass
        else:
            # store the link
            url_list.append(link)
            # webscrape the article body with beautiful soup
            response = requests.get(link)
            soup = BeautifulSoup(response.content)
            results=soup.find(id='content')
            body=results.find_all('div', class_='article-body__content')
            # save the results to the time and transcript lists
            transcript_list.append(body[0].text)
            time_list.append(soup.find('time').text)
            
  


In [5]:
df = pd.DataFrame(list(zip(url_list, time_list, transcript_list)), columns =['url', 'timestamp', 'transcript'])



In [6]:
df.to_csv('Maddow_transcripts.tsv', sep="\t")

